In [1]:
import torch
from omegaconf import OmegaConf
from data import pctDataset
from model import DistModel
from model import Trainer
from torch.utils.tensorboard import SummaryWriter

2025-04-02 21:43:52.143164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
conf = OmegaConf.load("config.yaml")
conf.ValLoaderParams = conf.LoaderParams
conf.ValLoaderParams.numWPTS = 2000
conf.ModelParams.particleNumber = conf.LoaderParams.ParticleNumber
conf.ModelParams.inDims = conf.LoaderParams.dims
conf.deviceNum = 2

ROOT_TO_DATA = '/home/bdudas/PCT_tracking/data'

In [3]:
trainDataset = pctDataset(ROOT_TO_DATA +"/train",**conf.LoaderParams)
trainLoader = torch.utils.data.DataLoader(trainDataset,batch_size=conf.TrainingParams.batch_size,shuffle=True)
valDataset = pctDataset(ROOT_TO_DATA +"/test",**conf.ValLoaderParams)
valLoader = torch.utils.data.DataLoader(valDataset,batch_size=conf.TrainingParams.batch_size)

In [4]:
batch = next(iter(trainLoader))
batch.shape

torch.Size([256, 100, 41, 3])

In [5]:
device = torch.device(f'cuda:{conf.deviceNum}' if torch.cuda.is_available() else 'cpu')
myModel = DistModel(**conf.ModelParams).to(device)
#myModel = torch.compile(myModel)

In [6]:
myModel(batch[:,:,0].to(device),batch[:,:,0].to(device))

tensor([[[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         ...,
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         ...,
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0099, 0.0099,  ..., 0.0101, 0.0101, 0.0100],
         [0.0100, 0.0101, 0.0101,  ..., 0.0099, 0.0099, 0.0100]],

        [[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.

In [7]:
myTrainer = Trainer(targetLayer=conf.TrainingParams.targetLayer,device=device)

In [8]:
conf.comment = ''

In [9]:
writer = SummaryWriter(comment= "_"+conf.comment)

In [10]:
with open(f'{writer.log_dir}/config.yaml', 'w') as f:
    OmegaConf.save(conf, f)

In [11]:
tmp = next(iter(trainLoader))
myModel(tmp[:,:,0].to(device),tmp[:,:,0].to(device))

tensor([[[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         ...,
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0101, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         ...,
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]],

        [[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
         [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.

In [12]:
myTrainer.train(model = myModel, loader = trainLoader,numEpochs=1#conf.TrainingParams.epochs
                ,valLoader = valLoader,writer=writer,optimizer=torch.optim.Adam,criterion=torch.nn.CrossEntropyLoss)

TypeError: 'module' object is not callable